# GPU Training for Carbon/Water Footprint Model
Train the XGBoost multi-output model with Colab GPU acceleration.

**Features (v2.0):**
- 67 features including formula features, target encoding, and interaction features
- Log-transformed targets for handling skewed distributions
- Robustness testing with missing value simulation

**Setup:** Run cells in order. Use GPU runtime for faster training.

## 1Fresh Clone (ALWAYS run this first!)

This ensures you have the latest code with all improvements.

In [ ]:
# ALWAYS run this cell first for a fresh start!
%cd /content

# Remove ANY old clone
!rm -rf /content/bulk_product_generator
!rm -rf /content/Avelero_HydroCarbon

# Clone fresh from GitHub
print("Cloning latest code from GitHub...")
!git clone https://github.com/Avelero/Avelero_HydroCarbon.git /content/bulk_product_generator

# Pull LFS files (CSV data files are stored in Git LFS)
%cd /content/bulk_product_generator
print("\nPulling LFS files (CSV data)...")
!git lfs install
!git lfs pull

# Show latest commit to verify we have newest code
print("\nLatest commit:")
!git log --oneline -3

# Verify CSV files are present
print("\nVerifying data files...")
!ls -lh data/data_splitter/output/*.csv 2>/dev/null || echo "WARNING: CSV files not found!"

# Install dependencies
%cd /content/bulk_product_generator/models
print("\nInstalling requirements...")
!pip install -q -r requirements.txt

print("\n[OK] Setup complete! Ready to train.")

/content
Cloning into '/content/bulk_product_generator'...
remote: Enumerating objects: 418, done.
remote: Counting objects: 100% (418/418), done.
remote: Compressing objects: 100% (315/315), done.
remote: Total 418 (delta 168), reused 330 (delta 84), pack-reused 0 (from 0)
Receiving objects: 100% (418/418), 5.92 MiB | 12.08 MiB/s, done.
Resolving deltas: 100% (168/168), done.
Filtering content: 100% (12/12), 1001.31 MiB | 72.36 MiB/s, done.
/content/bulk_product_generator/models


## 2. Check GPU

Verify GPU is available. Should show Tesla T4 or similar.

In [ ]:
# Check GPU availability
import torch
print(f" PyTorch CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f" GPU: {torch.cuda.get_device_name(0)}")

print("\n GPU Details:")
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

Wed Dec  3 16:33:45 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

## 3. Train Model

Choose between full training or quick test.

In [ ]:
# Configuration
QUICK_TEST = False  # Set to True for quick 10K sample test, False for full training

%cd /content/bulk_product_generator/models

if QUICK_TEST:
    print(" QUICK TEST MODE (10K samples)")
    print("=" * 50)
    !python train_max_accuracy.py --sample-size 10000 --save-dir saved/gpu_test
else:
    print(" FULL TRAINING MODE (all 450K+ samples)")
    print("=" * 50)
    print("Expected time: ~5-10 min training + ~3 min robustness testing")
    print()
    !python train_max_accuracy.py --save-dir saved/gpu_training

/content/bulk_product_generator/models
Random seed set to 42
2025-12-03 16:33:47,183 - main - INFO - ================================================================================
2025-12-03 16:33:47,183 - main - INFO - MAXIMUM ACCURACY TRAINING STRATEGY
2025-12-03 16:33:47,183 - main - INFO - ================================================================================
2025-12-03 16:33:47,183 - main - INFO - Priority: Maximum accuracy (cost is not a constraint)
2025-12-03 16:33:47,183 - main - INFO - Dataset: Full 676K samples
2025-12-03 16:33:47,183 - main - INFO - 
2025-12-03 16:33:47,183 - main - INFO - 
2025-12-03 16:33:47,183 - main - INFO - PHASE 1: BASELINE TRAINING (Complete Data - Maximum Accuracy)
2025-12-03 16:33:47,183 - main - INFO - ================================================================================
2025-12-03 16:33:47,183 - main - INFO - 
Step 1/5: Loading data...
Loading training data from /content/bulk_product_generator/data/data_splitter/output/trai

: 

## 4. View Results

In [ ]:
# View training summary
print(" TRAINING RESULTS")
print("=" * 60)

# Show final metrics from log
!grep -E "(R²|MAE|RMSE|carbon_|water_)" /content/bulk_product_generator/models/logs/training_max_accuracy.log | tail -20

print("\n" + "=" * 60)
print(" Saved files:")
!ls -la /content/bulk_product_generator/models/saved/gpu_training/ 2>/dev/null || echo "No saved files yet"

2025-12-03 16:09:55,470 - main - INFO - ================================================================================
2025-12-03 16:09:55,470 - main - INFO - MAXIMUM ACCURACY TRAINING STRATEGY
2025-12-03 16:09:55,470 - main - INFO - ================================================================================
2025-12-03 16:09:55,470 - main - INFO - Priority: Maximum accuracy (cost is not a constraint)
2025-12-03 16:09:55,470 - main - INFO - Dataset: Full 676K samples
2025-12-03 16:09:55,470 - main - INFO - 
2025-12-03 16:09:55,470 - main - INFO - 
2025-12-03 16:09:55,470 - main - INFO - PHASE 1: BASELINE TRAINING (Complete Data - Maximum Accuracy)
2025-12-03 16:09:55,471 - main - INFO - ================================================================================
2025-12-03 16:09:55,471 - main - INFO - 
Step 1/5: Loading data...
2025-12-03 16:10:01,539 - main - INFO - Step 2/5: Adding formula-based features...
2025-12-03 16:11:18,995 - main - INFO - Step 3/5: Preprocessing fea

##  Quick Reference

### Expected Results (with latest improvements):
| Target | Expected R² | Notes |
|--------|-------------|-------|
| carbon_transport | ~0.99 | Best performing |
| carbon_material | ~0.90+ | With target encoding |
| carbon_total | ~0.90+ | Sum of above |
| water_total | ~0.90+ | With target encoding |

### Script Options:
```bash
python train_max_accuracy.py [OPTIONS]

--sample-size N       Use N samples (default: all)
--save-dir DIR        Save directory
--skip-baseline       Skip baseline, only augmented
--force-augmentation  Force augmented retraining
--seed N              Random seed (default: 42)
```

### Troubleshooting:
- **Old results?** → Re-run Cell 1 (Fresh Clone)
- **GPU not found?** → Runtime > Change runtime type > GPU
- **Out of memory?** → Use `--sample-size 100000`

## 5. Download Model (Optional)

In [ ]:
# Download trained model and preprocessor
from google.colab import files
import shutil
import os

save_dir = "/content/bulk_product_generator/models/saved/gpu_training"

if os.path.exists(save_dir):
    # Create zip of saved model
    shutil.make_archive("/content/trained_model", 'zip', save_dir)
    print(" Created trained_model.zip")
    print(f"   Contents: {os.listdir(save_dir)}")
    
    # Download
    files.download("/content/trained_model.zip")
else:
    print("[ERROR] No saved model found. Run training first!")